# Import libraries

In [231]:
import pandas as pd
import sqlite3 
import json

connection = sqlite3.connect("cademycode.db")
cursor = connection.cursor()

# Connect to the database

In [233]:
connection = sqlite3.connect("cademycode.db")
cursor = connection.cursor()

# Get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables:")
for table in tables:
    print(table[0])

Tables:
cademycode_students
cademycode_courses
cademycode_student_jobs


In [235]:
#load the tables into dataframes

dfStudent = pd.read_sql_query("SELECT * FROM cademycode_students", connection)
dfCourses = pd.read_sql_query("SELECT * FROM cademycode_courses", connection)
dfStudentJobs = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", connection)

# Working with students_table


In [238]:
# use .head .info to explore and get info of the data starting with student
dfStudent.info()

print(dfStudent.head())

#obeservations from the studen table:
#contact ziet eruit als een json meer info is nodig om het in volledigheid te analyseren

#een selectie op de kolom is gemaakt, nog steeds niet alle informatie zichbaar
dfStudent2 = pd.read_sql_query("SELECT contact_info  FROM cademycode_students", connection)


#nu een selectie gemaakt op de eerste rij en kolom wat succesvol alles laat zien
first_contact_info = dfStudent2.iloc[0]['contact_info']
print(first_contact_info)
type(first_contact_info)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB
   uuid             name         dob sex  \
0     1  Annabelle Avery  1943-07-03   F   
1     2      Micah Rubio  1991-02-07   M   
2     3       Hosea Dale  1989-12-07   M   
3     4     Mariann Kirk  1988-07-31   F   
4     5  Lucio Alexander  1963-08-31   M   

     

str

In [314]:
dfStudent.current_career_path_id.unique()

array([ 1.,  8.,  9.,  3.,  6., 10.,  5.,  0.,  4.,  7.,  2.])

In [240]:
dfStudent.duplicated().sum()

0

In [242]:
#unit test
o = dfStudent['contact_info']
print(dfStudent['job_id'].dtype)

print(type(o))
print(o)
dfStudent.columns

#dfStudent[dfStudent['job_id'].isna()]

#dfStudent['current_career_path_id'][0], 0)


object
<class 'pandas.core.series.Series'>
0       {"mailing_address": "303 N Timber Key, Irondal...
1       {"mailing_address": "767 Crescent Fair, Shoals...
2       {"mailing_address": "P.O. Box 41269, St. Bonav...
3       {"mailing_address": "517 SE Wintergreen Isle, ...
4       {"mailing_address": "18 Cinder Cliff, Doyles b...
                              ...                        
4995    {"mailing_address": "591 Blue Berry, Coulee, I...
4996    {"mailing_address": "30 Iron Divide, Pewaukee ...
4997    {"mailing_address": "684 Rustic Rest Avenue, C...
4998    {"mailing_address": "900 Indian Oval, Euclid, ...
4999    {"mailing_address": "406 Zephyr Port, Oskaloos...
Name: contact_info, Length: 5000, dtype: object


Index(['uuid', 'name', 'dob', 'sex', 'contact_info', 'job_id',
       'num_course_taken', 'current_career_path_id', 'time_spent_hrs'],
      dtype='object')

# Observations student table

1. column contact info contains 2 variables: mailing_adress and email which will be split into multiple columns
2. data types the numerical columns are currently objects instead of floats or int and dob seems to be better suited as an date object
3. missing data: the columns job_id, num_course_taken, current_career_path_id, time_spent_hrs seems to have missing data more work is needed to determine the type of missing data and how to handle it  
4. dob making an age columns for future analyzing


In [245]:
#the column contact info is a string before we split it we are gonna make it a dict 

def dict_contact_info(contact_info):
    dict = json.loads(contact_info)
    return pd.Series(dict)

split_contact = dfStudent['contact_info'].apply(dict_contact_info)
split_contact.head()


303 N Timber Key, Irondale, Wisconsin, 84736


In [246]:
dfStudent['mailing_address']= split_contact['mailing_address']
dfStudent['email']= split_contact['email']
dfStudent = dfStudent.drop('contact_info', axis=1)
dfStudent.head()

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com


# Changing datatype

Changing the data types for the following columns:
1. UUID
2. job_id
3. num_course_taken
4. current_carreer_path_id
5. time_spent_hrs
6. dob


In [248]:
dfStudent = dfStudent.astype({
    'job_id': 'float',
    'num_course_taken': 'float',
    'current_career_path_id': 'float',
    'time_spent_hrs': 'float'
})

dfStudent['dob'] = pd.to_datetime(dfStudent['dob'])
dfStudent.info()
dfStudent['dob'].dtype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   sex                     5000 non-null   object        
 4   job_id                  4995 non-null   float64       
 5   num_course_taken        4749 non-null   float64       
 6   current_career_path_id  4529 non-null   float64       
 7   time_spent_hrs          4529 non-null   float64       
 8   mailing_address         5000 non-null   object        
 9   email                   5000 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 390.8+ KB


dtype('<M8[ns]')

# Handling missing data

checking the missing data to determine if the missing data is missing at random, structurally missing, or missing not at random:
1. job_id 
2. num_course_taken
3. current_career_path_id 
4. time_spent_hrs             

# job_id

In [250]:
dfStudent3 = pd.read_sql_query("SELECT * FROM cademycode_students WHERE job_id is NULL", connection)
dfStudent3.head(10)
#dfStudent.head(10)

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,163,Glen Riley,2002-08-22,M,"{""mailing_address"": ""P.O. Box 37267, Cornlea v...",None,8.0,3.0,5.7
1,758,Mercedez Vorberg,2002-03-25,F,"{""mailing_address"": ""284 Cedar Seventh, Virden...",None,15.0,4.0,4.14
2,855,Kurt Ho,2002-05-29,M,"{""mailing_address"": ""P.O. Box 27254, Olin, New...",None,0.0,8.0,23.72
3,1030,Penny Gaines,2002-03-01,N,"{""mailing_address"": ""138 Misty Vale, Stockton ...",None,15.0,4.0,16.25
4,1543,Frederick Reilly,2002-11-13,M,"{""mailing_address"": ""P.O. Box 40769, Quakervil...",None,7.0,9.0,21.32


In [251]:
dfStudent4 = pd.read_sql_query("SELECT * FROM cademycode_students WHERE dob like '2002%'", connection)
dfStudent4.head(15)

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,59,Tyron van Riemsdijk,2002-07-29,M,"{""mailing_address"": ""899 E Orchard, Estelline,...",8.0,12.0,8.0,6.17
1,133,Henry Estes,2002-07-25,F,"{""mailing_address"": ""357 Fallen Shadow, Seldov...",8.0,2.0,5.0,17.96
2,163,Glen Riley,2002-08-22,M,"{""mailing_address"": ""P.O. Box 37267, Cornlea v...",None,8.0,3.0,5.7
3,205,Kasey Conley,2002-06-02,N,"{""mailing_address"": ""172 S Mall, Streator, Nor...",8.0,5.0,5.0,16.95
4,265,Janna Abbott,2002-12-26,N,"{""mailing_address"": ""233 Stony Grove Divide, N...",8.0,6.0,2.0,4.39
5,295,Gidget Alford,2002-11-12,F,"{""mailing_address"": ""916 Cozy Fillmore Skyway,...",8.0,8.0,3.0,6.98
6,360,Felipe Wintershoven,2002-08-18,M,"{""mailing_address"": ""34 Rose Way, Harrah, Illi...",8.0,15.0,10.0,11.1
7,399,Blake Davidson,2002-02-11,M,"{""mailing_address"": ""816 Blue Ranch, Lismore, ...",8.0,2.0,10.0,12.99
8,549,Duncan Wu,2002-04-06,N,"{""mailing_address"": ""703 Dewy Divide, Hudson O...",8.0,9.0,2.0,13.3
9,630,Adrienne Herman,2002-11-02,F,"{""mailing_address"": ""P.O. Box 83603, Evant, Ca...",8.0,13.0,2.0,4.92


In [255]:
dfStudent = dfStudent.dropna(subset=["job_id"])
dfStudent.info()
dfStudent[dfStudent['job_id'].isna()]

<class 'pandas.core.frame.DataFrame'>
Index: 4995 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4995 non-null   int64         
 1   name                    4995 non-null   object        
 2   dob                     4995 non-null   datetime64[ns]
 3   sex                     4995 non-null   object        
 4   job_id                  4995 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4524 non-null   float64       
 7   time_spent_hrs          4524 non-null   float64       
 8   mailing_address         4995 non-null   object        
 9   email                   4995 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 429.3+ KB


,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email


For job_id what stands out is that they all have the dob 2002. upon further inspection it seems that other rows with the year 2002 have job_ids. Based of that i would have to put it in the category missing at random. with only 5 missing values deleting them would be the best course of action with minimal impact.

# num_course_taken

In [261]:
dfStudent4 = pd.read_sql_query("SELECT * FROM cademycode_students WHERE num_course_taken is NULL", connection)
dfStudent4.head(15)


,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,26,Doug Browning,1970-06-08,M,"{""mailing_address"": ""P.O. Box 15845, Devine, F...",7.0,None,5.0,1.92
1,27,Damon Schrauwen,1953-10-31,M,"{""mailing_address"": ""P.O. Box 84659, Maben, Ge...",4.0,None,10.0,3.73
2,52,Alisa Neil,1977-05-28,F,"{""mailing_address"": ""16 View Annex, Mosses, No...",5.0,None,8.0,22.86
3,71,Chauncey Hooper,1962-04-07,M,"{""mailing_address"": ""955 Dewy Flat, Slaughterv...",3.0,None,3.0,3.97
4,81,Ellyn van Heest,1984-06-28,F,"{""mailing_address"": ""872 Cider Glade, Chicken,...",3.0,None,10.0,12.39
5,97,Moises Krekel,2004-06-15,M,"{""mailing_address"": ""455 Silent Smith, Radium ...",8.0,None,None,None
6,140,Charlie Semvoort,2004-03-06,N,"{""mailing_address"": ""76 Dewy Quail, Whitesburg...",8.0,None,5.0,5.76
7,150,Lindsey Mcclain,1972-11-30,N,"{""mailing_address"": ""751 Robin Highlands, Hurl...",6.0,None,9.0,21.79
8,158,Ching Rijkensz,1988-02-17,N,"{""mailing_address"": ""471 Timber Arbor, Oaks, T...",3.0,None,7.0,2.36
9,203,Bettie Torres,1973-06-05,N,"{""mailing_address"": ""457 SW Valley, New Madiso...",6.0,None,9.0,6.04


In [263]:
dfStudent = dfStudent.dropna(subset=["num_course_taken"])
dfStudent.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4293 non-null   float64       
 7   time_spent_hrs          4293 non-null   float64       
 8   mailing_address         4744 non-null   object        
 9   email                   4744 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 407.7+ KB


looking at the none values no identifiable pattern is found. Even job_id had a pattern where the dob was constant at 2002. making this missing data at random. Making deleting the data the best course of action.


# current_career_path_id


In [267]:
dfStudent5 = pd.read_sql_query("SELECT * FROM cademycode_students WHERE current_career_path_id is NULL", connection)
dfStudent5.head(15)

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,16,Norene Dalton,1976-04-30,F,"{""mailing_address"": ""130 Wishing Essex, Branch...",6.0,0.0,None,None
1,20,Sofia van Steenbergen,1990-02-21,N,"{""mailing_address"": ""634 Clear Barn Dell, Beam...",7.0,13.0,None,None
2,31,Christoper Warner,1989-12-28,M,"{""mailing_address"": ""556 Stony Highlands, Drai...",2.0,5.0,None,None
3,50,Antony Horne,1996-05-29,M,"{""mailing_address"": ""P.O. Box 78685, Lenox, Te...",3.0,2.0,None,None
4,55,Omar Bunk,1955-11-08,M,"{""mailing_address"": ""445 Dale Hollow, Vermont ...",3.0,14.0,None,None
5,56,Lincoln Hartman,1950-07-15,M,"{""mailing_address"": ""P.O. Box 22581, Southmont...",5.0,8.0,None,None
6,63,Zana Willems,1953-11-23,F,"{""mailing_address"": ""319 Iron Treasure, Therma...",2.0,2.0,None,None
7,65,Jasmine Vos,1942-08-24,F,"{""mailing_address"": ""P.O. Box 56722, La Veta, ...",5.0,13.0,None,None
8,72,Genevie Huerta,1954-11-15,F,"{""mailing_address"": ""401 Quaking Smith, McDerm...",1.0,8.0,None,None
9,89,Scott de Boef,1976-01-21,M,"{""mailing_address"": ""P.O. Box 47591, Pemberton...",2.0,11.0,None,None


First observation is that whenever a current_career_path_id is missing time_spent_hrs also seems to be missing and looking at dfStudent.info() we see that they have the same values. This makes this structurally missing data. It seems to be the case and logical that if you do not have a carreer path you are also not spending any time. To handle this case of structurally missing data I will add an extra id to current_career_path_id which equals 'No career path' and set those hours to 0


First observation is that whenever a current_career_path_id is missing time_spent_hrs also seems to be missing and looking at

In [271]:
dfStudent['current_career_path_id'].unique()

array([ 1.,  8.,  9.,  3.,  6., 10.,  5., nan,  4.,  7.,  2.])

In [273]:
dfStudent['current_career_path_id'] = dfStudent['current_career_path_id'].fillna(value=0)
dfStudent['time_spent_hrs'] = dfStudent['time_spent_hrs'].fillna(value=0)
dfStudent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4744 non-null   float64       
 7   time_spent_hrs          4744 non-null   float64       
 8   mailing_address         4744 non-null   object        
 9   email                   4744 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 407.7+ KB


# Adding age column from dob
Adding an age column to make it easier for future analysis.

In [276]:
#Calculate age by subtracting birth year from current year, then adjust by subtracting 1 if the birthday hasn’t happened yet this year.
now = pd.to_datetime('now')
dfStudent['age'] = now.year - dfStudent['dob'].dt.year - (
    (now.month < dfStudent['dob'].dt.month) |
    ((now.month == dfStudent['dob'].dt.month) & (now.day < dfStudent['dob'].dt.day))
)
dfStudent.head()


,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email,age
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com,81
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.40,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com,34
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com,35
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com,36
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com,61


# working with cademycode_courses

In [279]:
dfCourses.head(11)

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


In [281]:
dfCourses.duplicated().sum()

0

In [283]:
dfCourses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes


# Observations cademycode_courses
This table shows no missing data, no incorrect data types and no need for data tidying. It does need to have an extra carreer path id of 0 added. The one I also added to the student table


In [298]:
dfCourses.loc[10] = [0,'No career path', 0]
dfCourses.head(11)


,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


# Working with student_jobs table

In [187]:
dfStudentJobs.head(15)

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


In [312]:
dfStudentJobs.duplicated().sum()

dfStudentJobs.duplicated()


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
11     True
12     True
dtype: bool

dfStudentJobs.info()

# Observations with student_jobs table
The only thing that needs to be adjusted on this table is removing the duplicate values of job id 3,4,5


In [193]:
dfStudentJobs = dfStudentJobs.drop_duplicates()
dfStudentJobs.duplicated().sum()

0

# Joining tables

In [202]:
final_df = dfStudent.merge(dfCourses, left_on='current_career_path_id', right_on='career_path_id', how='left')
final_df = final_df.merge(dfStudentJobs, on='job_id', how='left')
final_df.head()



,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email,age,career_path_id,career_path_name,hours_to_complete,job_category,avg_salary
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com,81,1,data scientist,20,HR,80000
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.40,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com,34,8,android developer,27,HR,80000
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com,35,8,android developer,27,HR,80000
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com,36,9,machine learning engineer,35,education,61000
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com,61,3,data analyst,12,HR,80000


In [200]:
final_df.info()
cursor.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4744 entries, 0 to 4743
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4744 non-null   float64       
 7   time_spent_hrs          4744 non-null   float64       
 8   mailing_address         4744 non-null   object        
 9   email                   4744 non-null   object        
 10  age                     4744 non-null   int32         
 11  career_path_id          4744 non-null   int64         
 12  career_path_name        4744 non-null   object  

# Create the Output CSV
Use the cleaned tables to produce an analytics-ready SQLite database and flat CSV file. The final CSV should contain all the data the analysts might need in a single table.

In [318]:
#rename cursor2 to connect2
cursor2 = sqlite3.connect('cademycode_ds.db')
final_df.to_sql('cademycode_aggregated', cursor2, if_exists='replace', index=False)

'4744'

In [320]:
str(len(final_df))

'4744'

In [210]:
db_df = pd.read_sql_query("SELECT * FROM cademycode_aggregated", cursor2)
db_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4744 entries, 0 to 4743
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uuid                    4744 non-null   int64  
 1   name                    4744 non-null   object 
 2   dob                     4744 non-null   object 
 3   sex                     4744 non-null   object 
 4   job_id                  4744 non-null   float64
 5   num_course_taken        4744 non-null   float64
 6   current_career_path_id  4744 non-null   float64
 7   time_spent_hrs          4744 non-null   float64
 8   mailing_address         4744 non-null   object 
 9   email                   4744 non-null   object 
 10  age                     4744 non-null   int64  
 11  career_path_id          4744 non-null   int64  
 12  career_path_name        4744 non-null   object 
 13  hours_to_complete       4744 non-null   int64  
 14  job_category            4744 non-null   

In [507]:
db_df.to_csv('cademycode_ds.csv')
cursor2.close()